# Data Preprocessing

Install Library

In [1]:
pip install swifter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 830 kB 4.2 MB/s 
     |████████████████████████████████| 281 kB 55.0 MB/s 
     |████████████████████████████████| 1.6 MB 54.9 MB/s 
  Created wheel for swifter: filename=swifter-1.3.4-py3-none-any.whl size=16322 sha256=f4993f28eba6d67687c0acb88d6c77072fd976bb818a6d42ce10fdfe495e5e0f
  Stored in directory: /root/.cache/pip/wheels/29/a7/0e/3a8f17ac69d759e1e93647114bc9bdc95957e5b0cbfd405205
Successfully built swifter
  Attempting uninstall: psutil
    Found existing installation: psutil 5.4.8
    Uninstalling psutil-5.4.8:
      Successfully uninstalled psutil-5.4.8


In [2]:
pip install openpyxl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
pip install Sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 209 kB 4.2 MB/s 


### Import Packages and Dataset

In [4]:
import pickle
import pandas as pd
import numpy as np
import re
import swifter
import matplotlib.pyplot as plt
import seaborn as sns
import time
import csv
from nltk.tag import CRFTagger
from nltk.tokenize import word_tokenize
from textblob import TextBlob
from random import random
from openpyxl import Workbook

sns.set_style("whitegrid")

from sklearn.model_selection import train_test_split

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

import warnings
warnings.filterwarnings("ignore")

In [5]:
# membuat akses ke google drive untuk load data

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [6]:
full_data_train = pd.read_csv('/content/drive/MyDrive/TA_MEIRA REYNITA PUTRI_1301164695/Code/Data/data_train_70_30_1.csv', encoding = 'unicode_escape', sep =',')
full_data_test = pd.read_csv('/content/drive/MyDrive/TA_MEIRA REYNITA PUTRI_1301164695/Code/Data/data_test_70_30_1.csv', encoding = 'unicode_escape', sep = ',')

In [7]:
full_data_train = full_data_train.reset_index(drop = True)
full_data_train.sample(5)

,Argumen,Kelas
1360,Inilah knp diksi consent di permendikbud dan u...,0
651,Permendikbud harus ditolak,1
1107,Uji coba permendikbud / terbukti berhasil meny...,1
1312,HTI ngotot banget menolak Permendikbud ampe bi...,-1
1074,Sudah tepat tindakan Nadiem yg ngeluarin perme...,1


In [8]:
full_data_test = full_data_test.reset_index(drop = True)
full_data_test.sample(5)

,Argumen,Kelas
571,yang pada naikin ht tolak permendikbud pada kn...,1
590,"Iya lah meningkat brarti orang"" yang pernah ke...",1
526,"ya, sama seperti keributan soal permendikbud. ...",1
27,"polemik permendikbud ini penting sebenarnya, b...",-1
391,Kejahatan seksual sering tjd krn relasi yg tim...,1


### Text Cleaning

1. Menghilangkan simbol khusus

In [9]:
def cleaning_data(data):
    data['Argumen_Cleaned_1'] = data['Argumen'].str.replace("\r", " ")
    data['Argumen_Cleaned_1'] = data['Argumen_Cleaned_1'].str.replace("\t", " ")
    data['Argumen_Cleaned_1'] = data['Argumen_Cleaned_1'].str.replace("\n", " ")
    return data

In [10]:
full_data_train = cleaning_data(full_data_train)
full_data_test = cleaning_data(full_data_test)

2. Ubah menjadi lowercase

In [11]:
def lowercase(data):
    data['Argumen_Cleaned_2'] = data['Argumen_Cleaned_1'].str.lower()
    return data

In [12]:
full_data_train = lowercase(full_data_train)
full_data_test = lowercase(full_data_test)

3. Menghilangkan emoji, simbol, dan tanda baca

In [13]:
RE_EMOJI = re.compile('[\U00010000-\U0010ffff]', flags=re.UNICODE)

def strip_emoji(text):
    return RE_EMOJI.sub(r'', text)

def remove_emoji(data):
    data['Argumen_Cleaned_3'] = data['Argumen_Cleaned_2']
    data['Argumen_Cleaned_3'] = data['Argumen_Cleaned_3'].map(lambda x: strip_emoji(x))
    return data

In [14]:
full_data_train = remove_emoji(full_data_train)
full_data_test = remove_emoji(full_data_test)

In [15]:
def remove_punct(data):
    data['Argumen_Cleaned_4'] = data['Argumen_Cleaned_3']
    data['Argumen_Cleaned_4'] = data['Argumen_Cleaned_4'].str.replace("[^a-zA-Z0-9]+", " ")
    data['Argumen_Cleaned_4'] = data['Argumen_Cleaned_4'].str.replace('\u2026', '')
    data['Argumen_Cleaned_4'] = data['Argumen_Cleaned_4'].str.replace("  ", " ")
    data['Argumen_Cleaned_4'] = data['Argumen_Cleaned_4'].str.replace("   ", " ")
    data['Argumen_Cleaned_4'] = data['Argumen_Cleaned_4'].str.replace("’s", "")
    data['Argumen_Cleaned_4'] = data['Argumen_Cleaned_4'].str.replace("'s", "")
    data['Argumen_Cleaned_4'] = data['Argumen_Cleaned_4'].str.replace("n't"," not")
    data['Argumen_Cleaned_4'] = data['Argumen_Cleaned_4'].str.replace("'re"," are")
    data['Argumen_Cleaned_4'] = data['Argumen_Cleaned_4'].str.replace("'tn"," nt")
    return data

In [16]:
full_data_train = remove_punct(full_data_train)
full_data_test = remove_punct(full_data_test)

4. Menghilangkan Angka

In [17]:
def removeNumber(data):
    data['Argumen_Cleaned_4'] = data['Argumen_Cleaned_4'].str.replace("[^a-zA-Z0-9]+", " ")
    return data

In [18]:
full_data_train = removeNumber(full_data_train)
full_data_test = removeNumber(full_data_test)

### Normalisasi Kata 

In [19]:
norm = pd.read_csv('/content/drive/MyDrive/TA_MEIRA REYNITA PUTRI_1301164695/Code/Data/Normalisasi_Kata.csv', encoding = 'unicode_escape')
norm.head()

,tak_baku,baku
0,thn,tahun
1,mmng,memang
2,bs,bisa
3,d,di
4,dkk,dan kawan kawan


In [20]:
kata_tak_baku = norm['tak_baku'].values.tolist()
kata_baku = norm['baku'].values.tolist()

In [21]:
def normalisasi_kata(kata, baku, tak_baku):
    kata = kata.split(" ")
    for j in range(len(kata)):
        if kata[j] in tak_baku:
            a = tak_baku.index(kata[j])
            kata[j] = baku[a]
    return ' '.join(map(str,kata))
    
def normalisasi_data(data, baku, tak_baku):
    data['Argumen_Normed'] = data['Argumen_Cleaned_4']
    data['Argumen_Normed'] = data['Argumen_Normed'].map(lambda x: normalisasi_kata(x,baku,tak_baku))
    return data

In [22]:
full_data_train = normalisasi_data(full_data_train, kata_baku, kata_tak_baku)
full_data_test = normalisasi_data(full_data_test, kata_baku, kata_tak_baku)

### Menghilangkan Stopword

In [23]:
factory_stop = StopWordRemoverFactory()
stopword = factory_stop.get_stop_words()

In [24]:
unwanted_list = ['namun', 'begitu', 'ok', 'oke', 'tidak', 'tetapi', 'tapi', 'amat', 'seharusnya', 'sebetulnya', 'setidaknya', 'tentu', 'bagaimanapun', 'daripada', 'sedangkan', 'sementara', 'agak', 'yang', 'dan', 'di', 'dari', 'nomor', 'tahun', 'no']
final_stopword = []

for element in stopword:
    if element not in unwanted_list:
        final_stopword.append(element)

In [25]:
def remove_stop(data):
    data['Argumen_Stopword'] = data['Argumen_Normed']

    for stop in final_stopword:
        re_stop = r" \b" + stop + r"\b "
        data['Argumen_Stopword'] = data['Argumen_Stopword'].str.replace(re_stop, ' ')
        # print(re_stop)
    return data

In [26]:
full_data_train = remove_stop(full_data_train)
full_data_test = remove_stop(full_data_test)

### Stemming Kata

In [27]:
factory_stem = StemmerFactory()
stemmer = factory_stem.create_stemmer()

In [28]:
def stemming_kata(kata):
    kata = kata.split(" ")
    for j in range(len(kata)):
        stem = [stemmer.stem(y) for y in kata]
    return ' '.join(map(str,stem))

def stemming_data(data):
    data['Argumen_Stemmed'] = data['Argumen_Stopword']
    data['Argumen_Stemmed'] = data['Argumen_Stemmed'].swifter.apply(lambda x: stemming_kata(x))
    return data

In [29]:
full_data_train = stemming_data(full_data_train)
full_data_test = stemming_data(full_data_test)

Pandas Apply:   0%|          | 0/1561 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/669 [00:00<?, ?it/s]

### Fixing Data

In [30]:
def fixData(data):
      data_fix = data.drop(['Argumen_Cleaned_1', 'Argumen_Cleaned_2', 'Argumen_Cleaned_3', 'Argumen_Cleaned_4', 'Argumen_Normed', 'Argumen_Stopword'], axis=1)
      data_fix = data_fix.rename(columns={'Argumen_Stemmed': 'Argumen_Fixed'})
      return data_fix

In [31]:
full_data_train = fixData(full_data_train)
full_data_test = fixData(full_data_test)

### Save Data

In [32]:
full_data_train.to_csv('/content/drive/MyDrive/TA_MEIRA REYNITA PUTRI_1301164695/Code/Data/full_data_train_1.csv')
full_data_test.to_csv('/content/drive/MyDrive/TA_MEIRA REYNITA PUTRI_1301164695/Code/Data/full_data_test_1.csv')